# News Article Summarization. 
This notebook prepares news articles to be inserted into the ChromaDB vector database. First, it creates a summary of the news article, then it identifies important Named Entities such as the names of politicians, locations, and relevant dates. Having a news summary allows us to quickly perform small-to-big retrieval; finding the full article from it's brief overview. This method helps us evaluate the Language Learning Model (LLM) more effectively. Additionally, the identified entities will be used as metadata and embedded with the news articles, aiding in fine-tuning and evaluating the LLM.

For more insight into this approach, check out this YouTube video by Jerry Liu, Founder of LlmamaIndex: https://youtu.be/TRjq7t2Ms5I.

## Config & Install Libraries
Check if Huggingface transformers and required libraries are installed

In [ ]:
!pip install -q transformers sentencepiece sentence-transformers datasets spacy chromadb

In [ ]:
!python -m spacy download en_core_web_sm

## News Summary Pipeline

In [ ]:
import os
import json
from util import utils
from pymongo import MongoClient
from dotenv import load_dotenv, find_dotenv

In [ ]:
load_dotenv(find_dotenv())

### Parameters

In [ ]:
collection_name = 'raw-news'
batch_date = {'$gte': '2024-06-04', '$lte': '2024-06-05'}

In [ ]:
MONGO_CONN_STRING = os.getenv("MONGO_CONNECTION_STRING")
DB_NAME = os.getenv("DB_NAME")

In [ ]:
mongo_client = MongoClient(MONGO_CONN_STRING)
db = mongo_client.get_database(DB_NAME)

### Prepare Dataset

In [ ]:
news_articles = json.loads(json.dumps(list(db.get_collection(collection_name).find({'created_at': batch_date})), cls=utils.CustomMongoDecoder))

In [ ]:
for article in news_articles:
    article['processed_content'] = ''.join(art.strip() for art in article['raw_content'])
    article['processed_content'] = article['processed_content'].replace('\xa0', ' ')

In [ ]:
def split_text(text, max_length=1000, separator='\n\n'):
    chunks = []
    current_chunk = []
    for line in text.split('.'):
        if len('.'.join(current_chunk)) + len(line) < max_length:
            current_chunk.append(line)
        else:
            chunks.append('.'.join(current_chunk))
            current_chunk = [line]
    if current_chunk:
        chunks.append('.'.join(current_chunk))
    return separator.join(chunks)

In [ ]:
split_text(article['processed_content'])

## Load Summarization Model

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

# Load fine-tuned BART model for summarization
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

separator='\n\n'

def summarize_article(text: str):
    chunks = split_text(text, max_length=1000)
    summaries = []
    for chunk in chunks.split(separator):
        inputs = tokenizer([chunk], return_tensors="pt", truncation=True, max_length=1024)
        summary_ids = model.generate(inputs.input_ids, num_beams=4, length_penalty=2.0, early_stopping=True)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summaries.append(summary)
    return ' '.join(summaries)

In [ ]:
sample_summary = summarize_article(article['processed_content'])

In [ ]:
sample_summary

### Named Entity Recognition

In [ ]:
REQUIRED_FIELDS = ['PERSON', 'GPE', 'NORP', 'EVENT', 'ORG']

In [ ]:
import spacy
from collections import defaultdict

def perform_ner(text: str):
    
    # Load the English language model
    nlp = spacy.load("en_core_web_sm")
    
    # Process the text with spaCy
    doc = nlp(text)
    
    # Extract named entities
    entities = [(ent.text, ent.label_) for ent in doc.ents]

    return entities

In [ ]:
# postprocess the named entities to select the required entity tags
def postprocess_entities(entities):
    processed_entities = defaultdict(set)
    
    for entity, label in entities:
        if label in REQUIRED_FIELDS:
            processed_entities[label].add(entity)
    processed_entities = {key: list(value) for key, value in processed_entities.items()}
    return processed_entities

## Perform Summarization and NER on News Articles

In [ ]:
from bson import ObjectId

In [ ]:
for news in news_articles:
    summary = summarize_article(news['processed_content'])
    entities = postprocess_entities(perform_ner(news['processed_content']))

    news['news_summary'] = summary
    news['entities'] = entities

    # filter criteria
    filter_criteria = {'_id': ObjectId(news['_id'])}
    
    # Define the update operation
    update_data = {
        '$set': {
            'processed_content': news['processed_content'],
            'news_summary': summary,
            'entities': entities
        }
    }
    
    # Update the Mongo document
    result = db.get_collection('raw-news').update_one(filter_criteria, update_data)

## Save content and metadata on Chromadb

In [ ]:
import chromadb
import numpy as np
from chromadb.utils import embedding_functions
from langchain_mistralai.embeddings import MistralAIEmbeddings

In [ ]:
CHROMA_DB_URL = os.getenv('CHROMA_DB_URL')

In [ ]:
chroma_client = chromadb.HttpClient(CHROMA_DB_URL)

In [ ]:
try:
    collection = chroma_client.get_collection(DB_NAME)
except:
    collection = chroma_client.create_collection(DB_NAME)

In [ ]:
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

In [ ]:
emb = sentence_transformer_ef([article['processed_content']])

In [ ]:
assert np.array(emb).shape[0] == 1 

In [ ]:
for news in news_articles:
    collection.add(
        documents=[news['processed_content']],
        embeddings=sentence_transformer_ef([news['processed_content']]),
        metadatas=[{'entities': json.dumps(news['entities']), 
                    'summary': news['news_summary'], 
                    'source': news['source'],
                    'publication_date': news['publication_date']
                   }],
        ids=[str(news['_id'])]
    )

### Test Chromadb Querying

In [ ]:
TEST_QUERY = """
    What's the latest in Texas?
"""

In [ ]:
query_entities = postprocess_entities(perform_ner(TEST_QUERY))
query_embeddings = sentence_transformer_ef([TEST_QUERY])

In [ ]:
query_entities

In [ ]:
collection.query(query_embeddings=query_embeddings, n_results=1)

In [ ]:
len(collection.query(query_embeddings=query_embeddings, n_results=1)['ids'])

## LangChain

In [ ]:
!pip install langchain langchain-chroma langchain_community

In [ ]:
import chromadb
from langchain_chroma import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

In [ ]:
chroma_client = chromadb.HttpClient(CHROMA_DB_URL)

In [ ]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
langchain_chroma = Chroma(
    client=chroma_client,
    collection_name=DB_NAME,
    embedding_function=embedding_function,
)

In [ ]:
print("There are", langchain_chroma._collection.count(), "items in the collection")

In [ ]:
query = "How do you think michigan will vote this coming election"
docs = langchain_chroma.similarity_search(query)
print(docs[0].page_content)